In [6]:
import pandas as pd

In [7]:
game_data = pd.read_csv('../../data/cleaned/NFL_wk_by_wk_cleaned.csv')
modeling_file = pd.read_csv('../../data/modeling/modeling_file.csv')


In [8]:
game_data.head()

,WEEK_NUM,GAME_DAY_OF_WEEK,GAME_DATE,GAMETIME,WINNER,GAME_LOCATION,LOSER,BOXSCORE_LINK,PTS_WINNER,PTS_LOSER,YARDS_WINNER,TURNOVERS_WINNER,YARDS_LOSER,TURNOVERS_LOSER,YEAR,WIN_OR_TIE
0,1,Sun,1991-09-01,1:00PM,Kansas City Chiefs,NULL_VALUE,Atlanta Falcons,/boxscores/199109010kan.htm,14,6,291,1,243,6,91-92,1.0
1,1,Sun,1991-09-01,1:00PM,Dallas Cowboys,@,Cleveland Browns,/boxscores/199109010cle.htm,26,0,395,0,275,0,91-92,1.0
2,1,Sun,1991-09-01,1:00PM,Philadelphia Eagles,@,Green Bay Packers,/boxscores/199109010gnb.htm,20,5,318,3,228,5,91-92,1.0
3,1,Sun,1991-09-01,1:00PM,New Orleans Saints,NULL_VALUE,Seattle Seahawks,/boxscores/199109010nor.htm,27,4,244,3,348,4,91-92,1.0
4,1,Sun,1991-09-01,1:00PM,New York Jets,NULL_VALUE,Tampa Bay Buccaneers,/boxscores/199109010nyj.htm,16,2,271,0,277,2,91-92,1.0


In [9]:
game_data = game_data.loc[:,['WINNER','LOSER','YEAR','WEEK_NUM','GAME_LOCATION']]


In [10]:
game_data[game_data.duplicated(keep=False)]

,WINNER,LOSER,YEAR,WEEK_NUM,GAME_LOCATION


In [11]:

## Reduce data to just columns we want to look at here
game_data = game_data.loc[:,['WINNER','LOSER','YEAR','WEEK_NUM','GAME_LOCATION']]

## Change value of GAME_LOCATION to be 0 for home games or 1 for away games
game_data['GAME_LOCATION'] = game_data['GAME_LOCATION'].apply(lambda x: 0 if x=='NULL_VALUE' else 1)

## Create a 'flipped' version of game_data just swapping the winner and loser column
game_data_flipped = game_data.loc[:,['LOSER','WINNER','YEAR','WEEK_NUM','GAME_LOCATION']]
game_data_flipped['GAME_LOCATION'] =  game_data_flipped['GAME_LOCATION'].apply(lambda x: 0 if x==1 else 1)

game_data.columns         = ['TEAM_A','TEAM_B','YEAR','WEEK_NUM','GAME_LOCATION']
game_data_flipped.columns = ['TEAM_A','TEAM_B','YEAR','WEEK_NUM','GAME_LOCATION']

game_data = pd.concat([game_data, game_data_flipped])

In [12]:
game_data.shape

(16730, 5)

In [13]:
modeling_file.shape

(16730, 7)

In [14]:
modeling_file = modeling_file.merge(game_data, how='left', left_on=['TEAM_A','TEAM_B','YEAR','WEEK_NUM'], right_on=['TEAM_A','TEAM_B','YEAR','WEEK_NUM'])

In [19]:
modeling_file

,TEAM_A,TEAM_B,YEAR,WEEK_NUM,RESULT,TEAM_A_ELO,TEAM_B_ELO,GAME_LOCATION
0,Arizona Cardinals,Los Angeles Rams,91-92,1,1,1300.000000,1300.000000,1
1,Atlanta Falcons,Kansas City Chiefs,91-92,1,0,1300.000000,1300.000000,1
2,Buffalo Bills,Miami Dolphins,91-92,1,1,1300.000000,1300.000000,0
3,Chicago Bears,Minnesota Vikings,91-92,1,1,1300.000000,1300.000000,0
4,Cincinnati Bengals,Denver Broncos,91-92,1,0,1300.000000,1300.000000,1
...,...,...,...,...,...,...,...,...
16725,Kansas City Chiefs,Cincinnati Bengals,22-23,21,1,1460.175343,1400.943949,0
16726,Philadelphia Eagles,San Francisco 49ers,22-23,21,1,1380.498727,1419.524041,0
16727,San Francisco 49ers,Philadelphia Eagles,22-23,21,0,1419.524041,1380.498727,1
16728,Kansas City Chiefs,Philadelphia Eagles,22-23,22,1,1468.764144,1391.432035,1


In [15]:
game_data[game_data.duplicated(keep=False)]

,TEAM_A,TEAM_B,YEAR,WEEK_NUM,GAME_LOCATION


In [16]:
modeling_file[modeling_file.duplicated(keep=False)]

,TEAM_A,TEAM_B,YEAR,WEEK_NUM,RESULT,TEAM_A_ELO,TEAM_B_ELO,GAME_LOCATION


In [18]:
game_data.loc[game_data['TEAM_A']=='Atlanta Falcons',:].sort_values(['YEAR','WEEK_NUM'])

,TEAM_A,TEAM_B,YEAR,WEEK_NUM,GAME_LOCATION
2206,Atlanta Falcons,San Francisco 49ers,00-01,1,0
2229,Atlanta Falcons,Denver Broncos,00-01,2,1
2234,Atlanta Falcons,Carolina Panthers,00-01,3,1
2249,Atlanta Falcons,Los Angeles Rams,00-01,4,0
2273,Atlanta Falcons,Philadelphia Eagles,00-01,5,1
...,...,...,...,...,...
2124,Atlanta Falcons,New Orleans Saints,99-00,13,0
2143,Atlanta Falcons,San Francisco 49ers,99-00,14,1
2149,Atlanta Falcons,Tennessee Titans,99-00,15,1
2170,Atlanta Falcons,Arizona Cardinals,99-00,16,0


In [8]:
game_data.head()

,WINNER,LOSER,YEAR,WEEK_NUM,GAME_LOCATION
0,Kansas City Chiefs,Atlanta Falcons,91-92,1,0
1,Dallas Cowboys,Cleveland Browns,91-92,1,1
2,Philadelphia Eagles,Green Bay Packers,91-92,1,1
3,New Orleans Saints,Seattle Seahawks,91-92,1,0
4,New York Jets,Tampa Bay Buccaneers,91-92,1,0


In [9]:
game_data_flipped.head()

,LOSER,WINNER,YEAR,WEEK_NUM,GAME_LOCATION
0,Atlanta Falcons,Kansas City Chiefs,91-92,1,1
1,Cleveland Browns,Dallas Cowboys,91-92,1,0
2,Green Bay Packers,Philadelphia Eagles,91-92,1,0
3,Seattle Seahawks,New Orleans Saints,91-92,1,1
4,Tampa Bay Buccaneers,New York Jets,91-92,1,1
